In [129]:
import os
import re
import sys
import json
import requests
import logging
import traceback
import pdfplumber
import numpy as np
import pandas as pd
import logging.config
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [130]:
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
driver = webdriver.Chrome(options=chrome_options)
from pathlib import Path
path_to_download_folder = str(os.path.join(Path.home(), "Downloads"))
cik_length = 10

In [161]:
with open(r'/Users/ivan/Desktop/ESGenius/config/name_to_cik.json', 'r') as file:
    data = json.load(file)

cik_list = [entry['CIK'].zfill(10) for entry in data]
cik_list[:10]

['0001090872',
 '0000320193',
 '0001551152',
 '0001559720',
 '0000001800',
 '0000947484',
 '0001467373',
 '0000796343',
 '0000006281',
 '0000007084']

In [134]:
url = f"https://www.sec.gov/edgar/search/#/category=custom&entityName={cik_number}&forms=10-K"
driver.get(url)

In [137]:
tbody_html = driver.find_element(By.ID, 'results-grid').get_attribute('innerHTML')
soup = BeautifulSoup(tbody_html, 'html.parser')
filetype_elements = soup.select('td.filetype')
data_adsh_list = [element.find('a')['data-adsh'] for element in filetype_elements if element.find('a')]
data_adsh_list

['0001090872-23-000020',
 '0001090872-22-000026',
 '0001090872-21-000027',
 '0001090872-20-000020',
 '0001090872-19-000022']

In [ ]:
accession_number = '0001090872-23-000020'
accession_number.split('-')[0]
index_key = accession_number.split('-')[0]
accession_number_without_zeros = index_key.lstrip('0')
accession_number_without_dashes = accession_number.replace('-', '')

In [ ]:
data_file_url = f"https://www.sec.gov/Archives/edgar/data/{accession_number_without_zeros}/{accession_number_without_dashes}/{accession_number}.txt"
driver.get(data_file_url)

In [ ]:
body_content = driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML')
soup = BeautifulSoup(body_content, 'html.parser')

In [146]:
file_name = data_file_url.split("/")[-1]
download_path = os.path.join(path_to_download_folder, file_name)
download_path

'/Users/ivan/Downloads/0001090872-23-000020.txt'

In [154]:
with open('{}'.format(download_path), mode='wt', encoding='utf-8') as file:
    file.write(soup.text)

In [155]:
driver.quit()